In [1]:
import os
import torch
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import KFold, StratifiedKFold
from contextlib import contextmanager
import time

In [2]:
data_path = '/home/wdx/Downloads/severstal-steel-defect-detection'
walk = os.walk(data_path)
main_dir_path, subdirs_path, csv_path = next(walk)
print(main_dir_path)
print(subdirs_path)
print(csv_path)
# dir_im_path, _, im_path = next(walk)
# print(dir_im_path)
# print(im_path)
train_csv_path = os.path.join(main_dir_path, csv_path[0])
test_csv_path = os.path.join(main_dir_path, csv_path[1])
df = pd.read_csv(train_csv_path)
# some preprocessing
# https://www.kaggle.com/amanooo/defect-detection-starter-u-net
df['ImageId'], df['ClassId'] = zip(*df['ImageId_ClassId'].str.split('_'))
df['ClassId'] = df['ClassId'].astype(int)
df = df.pivot(index='ImageId', columns='ClassId', values='EncodedPixels')
df['defects'] = df.count(axis=1)
df.head(10)

/home/wdx/Downloads/severstal-steel-defect-detection
['train_images', 'test_images']
['train.csv', 'sample_submission.csv']


ClassId,1,2,3,4,defects
ImageId,,,,,
0002cc93b.jpg,29102 12 29346 24 29602 24 29858 24 30114 24 3...,NaN,NaN,NaN,1
00031f466.jpg,NaN,NaN,NaN,NaN,0
000418bfc.jpg,NaN,NaN,NaN,NaN,0
000789191.jpg,NaN,NaN,NaN,NaN,0
0007a71bf.jpg,NaN,NaN,18661 28 18863 82 19091 110 19347 110 19603 11...,NaN,1
000a4bcdd.jpg,37607 3 37858 8 38108 14 38359 20 38610 25 388...,NaN,NaN,NaN,1
000f6bf48.jpg,NaN,NaN,NaN,131973 1 132228 4 132483 6 132738 8 132993 11 ...,1
0014fce06.jpg,NaN,NaN,229501 11 229741 33 229981 55 230221 77 230468...,NaN,1
001982b08.jpg,NaN,NaN,NaN,NaN,0


In [3]:
sum(df['defects'].values == 0)

5902

In [4]:
sum(df['defects'].values == 1)

6239

In [5]:
sum(df['defects'].values == 2)

425

In [6]:
sum(df['defects'].values == 3)

2

In [7]:
sum(df['defects'].values == 4)

0

In [8]:
len(df)

12568

In [9]:
df.count(axis=0)


ClassId
1            897
2            247
3           5150
4            801
defects    12568
dtype: int64

In [10]:
df10 = df.head(12)
df10['defects'].values
df10.iloc[0]


ClassId
1          29102 12 29346 24 29602 24 29858 24 30114 24 3...
2                                                        NaN
3                                                        NaN
4                                                        NaN
defects                                                    1
Name: 0002cc93b.jpg, dtype: object

In [11]:
labels = df10.iloc[0][:4]
labels

ClassId
1    29102 12 29346 24 29602 24 29858 24 30114 24 3...
2                                                  NaN
3                                                  NaN
4                                                  NaN
Name: 0002cc93b.jpg, dtype: object

In [12]:
labels.values

array(['29102 12 29346 24 29602 24 29858 24 30114 24 30370 24 30626 24 30882 24 31139 23 31395 23 31651 23 31907 23 32163 23 32419 23 32675 23 77918 27 78174 55 78429 60 78685 64 78941 68 79197 72 79452 77 79708 81 79964 85 80220 89 80475 94 80731 98 80987 102 81242 105 81498 105 81754 104 82010 104 82265 105 82521 31 82556 69 82779 27 82818 63 83038 22 83080 57 83297 17 83342 50 83555 13 83604 44 83814 8 83866 37 84073 3 84128 31 84390 25 84652 18 84918 8 85239 10 85476 29 85714 47 85960 57 86216 57 86471 58 86727 58 86983 58 87238 59 87494 59 87750 59 88005 60 88261 60 88517 60 88772 61 89028 53 89283 40 89539 32 89667 10 89795 30 89923 28 90050 29 90179 37 90306 27 90434 38 90562 14 90690 38 90817 9 90946 38 91073 3 91202 38 91458 38 91714 38 91969 39 92225 39 92481 39 92737 39 92993 39 93248 40 93504 40 93760 40 94026 30 94302 10 189792 7 190034 21 190283 28 190539 28 190795 28 191051 28 191307 28 191563 28 191819 28 192075 28 192331 28 192587 28 192843 23 193099 14 193355 5',
    

In [13]:
df10

ClassId,1,2,3,4,defects
ImageId,,,,,
0002cc93b.jpg,29102 12 29346 24 29602 24 29858 24 30114 24 3...,NaN,NaN,NaN,1
00031f466.jpg,NaN,NaN,NaN,NaN,0
000418bfc.jpg,NaN,NaN,NaN,NaN,0
000789191.jpg,NaN,NaN,NaN,NaN,0
0007a71bf.jpg,NaN,NaN,18661 28 18863 82 19091 110 19347 110 19603 11...,NaN,1
000a4bcdd.jpg,37607 3 37858 8 38108 14 38359 20 38610 25 388...,NaN,NaN,NaN,1
000f6bf48.jpg,NaN,NaN,NaN,131973 1 132228 4 132483 6 132738 8 132993 11 ...,1
0014fce06.jpg,NaN,NaN,229501 11 229741 33 229981 55 230221 77 230468...,NaN,1
001982b08.jpg,NaN,NaN,NaN,NaN,0


In [14]:
nfold = 4
seed = 11
kf = KFold(n_splits=nfold,
           shuffle=True,
           random_state=seed)

loaders = []
idx = []
for train_ids, val_ids in kf.split(df10.index, df10['defects'].values):
    print(train_ids)
    print(val_ids)
    print("------")
    
df10.iloc[val_ids]

[ 0  1  2  4  5  7  9 10 11]
[3 6 8]
------
[0 1 3 4 5 6 7 8 9]
[ 2 10 11]
------
[ 0  1  2  3  6  8  9 10 11]
[4 5 7]
------
[ 2  3  4  5  6  7  8 10 11]
[0 1 9]
------


ClassId,1,2,3,4,defects
ImageId,,,,,
0002cc93b.jpg,29102 12 29346 24 29602 24 29858 24 30114 24 3...,NaN,NaN,NaN,1
00031f466.jpg,NaN,NaN,NaN,NaN,0
001d1b355.jpg,NaN,NaN,NaN,NaN,0


In [15]:
samples = np.random.choice(df10.index, 1)
samples[0]
len(df10)

12

In [16]:
df['defects']

ImageId
0002cc93b.jpg    1
00031f466.jpg    0
000418bfc.jpg    0
000789191.jpg    0
0007a71bf.jpg    1
                ..
fff0295e1.jpg    0
fff02e9c5.jpg    1
fffe98443.jpg    1
ffff4eaa8.jpg    1
ffffd67df.jpg    1
Name: defects, Length: 12568, dtype: int64

In [17]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

In [18]:
for i in range(10):
    with timer('Fold {}'.format(i)):
        if i < 4:
            continue
        print(i)

Fold 0 - done in 0s
Fold 1 - done in 0s
Fold 2 - done in 0s
Fold 3 - done in 0s
4
Fold 4 - done in 0s
5
Fold 5 - done in 0s
6
Fold 6 - done in 0s
7
Fold 7 - done in 0s
8
Fold 8 - done in 0s
9
Fold 9 - done in 0s


In [19]:
for i, metric in enumerate(['loss', 'iou', 'mAP']):
    print('train_{}_{}'.format(metric, i))

train_loss_0
train_iou_1
train_mAP_2


In [20]:
a = np.random.randn(4, 3, 2, 4)
a.shape
b = a[1, :, :,:]
b.shape

(3, 2, 4)

In [21]:
df['ImageId'], df['ClassId'] = zip(*df['ImageId_ClassId'].str.split('_'))
df['ClassId'] = df['ClassId'].astype(int)
df = df.pivot(index='ImageId', columns='ClassId', values='EncodedPixels')
df['defects'] = df.count(axis=1)
df.head(10)

KeyError: 'ImageId_ClassId'

In [ ]:
df_test = pd.read_csv(test_csv_path)
df_test.head(10)

In [ ]:
df_test['ImageId'] = df_test['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
df_test.head(10)

In [ ]:
df_tt = df_test['ImageId'].unique().tolist()
df_tt